<a href="https://colab.research.google.com/github/Hrithik2212/Neural-Machine-Translation--English-to-Tamil/blob/main/03_NMT_with_Attention_Tensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [90]:
#Libraries
import pathlib
import random
import string
import re
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import TextVectorization
from tensorflow.keras.callbacks import EarlyStopping


In [4]:
eng_file = '/content/drive/MyDrive/en-ta/train.en'
tamil_file = '/content/drive/MyDrive/en-ta/train.ta'

In [5]:
with open(eng_file, 'r') as file:
    english_sentences = file.readlines()
with open(tamil_file, 'r') as file:
    tamil_sentences = file.readlines()


english_sentences = [sentence.rstrip('\n').lower() for sentence in english_sentences]
tamil_sentences = [sentence.rstrip('\n') for sentence in tamil_sentences]

In [6]:
# filter out sentences which contains letters other than whats listed below
tamil_vocabulary = [ ' ', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/',
                      '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', '<', '=', '>', '?', 'ˌ',
                      "ஃ", "அ", "ஆ", "இ", "ஈ", "உ", "ஊ", "எ", "ஏ", "ஐ", "ஒ", "ஓ", "ஔ",
                      "க", "ங", "ச", "ஜ", "ஞ", "ட", "ண",
                      "த", "ந", "ன", "ப", "ம", "ய", "ர",
                      "ற", "ல", "ள", "ழ", "வ", "ஶ", "ஷ", "ஸ", "ஹ",
                      "ா", "ி", "ீ", "ு", "ூ", "ெ", "ே", "ை", "ொ", "ோ", "ௌ", "்", "ௗ",
                      "௦", "௧", "௨", "௫", "௬", "௲", "௳"
                      ]

english_vocabulary = [' ', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/',
                        '0', '1', '2', '3', '4', '5', '6', '7', '8', '9',
                        ':', '<', '=', '>', '?', '@',
                        '[', '\\', ']', '^', '_', '`',
                        'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l',
                        'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x',
                        'y', 'z',
                        '{', '|', '}', '~']

In [7]:
def is_valid_tokens(sentence , vocab):
    for token in list(set(sentence)):
        if token not in vocab:
            return False
    return True

valid_sentence_indicies = []
for index in range(len(english_sentences[:150000])):
    tamil_sentence, english_sentence = tamil_sentences[index], english_sentences[index]
    if is_valid_tokens(tamil_sentence, tamil_vocabulary) \
      and is_valid_tokens(english_sentence , english_vocabulary):
        valid_sentence_indicies.append(index)

In [8]:
TOTAL_SENTENCES = 50000 # lets take 50000 valid sentences to train the model
x = [english_sentences[i] for i in valid_sentence_indicies[:TOTAL_SENTENCES]]
y = [tamil_sentences[i] for i in valid_sentence_indicies[:TOTAL_SENTENCES]]

In [10]:
import pandas as pd
dataset =  pd.DataFrame({"English":x , "Tamil":y })
dataset.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   English  50000 non-null  object
 1   Tamil    50000 non-null  object
dtypes: object(2)
memory usage: 781.4+ KB


In [11]:
dataset.sample(5)

,English,Tamil
49638,pm modi spends diwali with army jawans,தீபாவளிப் பண்டிகையை ராணுவ வீரர்களுடன் கொண்டாடு...
27546,"""""""why so much celebration?""","""""""இவ்வளவு பெரிய விழா நடத்தறீங்க?"""
29531,both are happy.,இருவருக்கும் மகிழ்ச்சி.
4695,i dont get time.,எனக்கு நேரம் போவது தெரியவில்லை.
18395,take vinegar in a pan.,ஒரு பாத்திரத்தில் வெல்லத்தை எடுத்துக்கொள்ளவும்.


In [12]:
text_pairs = []

for i in range(dataset.shape[0]) :
    eng = str(dataset["English"][i])
    tam = "[start] " + str(dataset["Tamil"][i]) + " [end]"
    text_pairs.append((eng,tam))

In [13]:
len(text_pairs)


50000

In [14]:
random.shuffle(text_pairs) #Shuffle the pairs
train_sample_size = len(text_pairs) - int(0.2*len(text_pairs))
train_pairs = text_pairs[:train_sample_size]
test_pairs = text_pairs[train_sample_size:]

In [15]:
print("Total pairs :", len(text_pairs))
print("Train pairs :", len(train_pairs))
print("Test pairs :", len(test_pairs))

Total pairs : 50000
Train pairs : 40000
Test pairs : 10000


In [95]:
strip_chars = string.punctuation + "|"
strip_chars = strip_chars.replace("[", "")
strip_chars = strip_chars.replace("]", "")
strip_chars = strip_chars.replace('""', "")

vocab_size = 50000
sequence_length = 30
batch_size = 64

In [96]:
def custom_standardization(input_string):
    lowercase = tf.strings.lower(input_string)
    return tf.strings.regex_replace(lowercase, "[%s]" % re.escape(strip_chars), "")

In [97]:
eng_vectorization = TextVectorization(
    max_tokens=vocab_size, output_mode="int", output_sequence_length=sequence_length,
)

tam_vectorization = TextVectorization(
    max_tokens=vocab_size,
    output_mode="int",
    output_sequence_length=sequence_length + 1,
    standardize=custom_standardization,
)

In [98]:


train_eng_texts = [pair[0] for pair in train_pairs]
train_tam_texts = [pair[1] for pair in train_pairs]
eng_vectorization.adapt(train_eng_texts)
tam_vectorization.adapt(train_tam_texts)

In [99]:


def format_dataset(eng, tam):
    eng = eng_vectorization(eng)
    tam = tam_vectorization(tam)
    return ({"encoder_inputs": eng, "decoder_inputs": tam[:, :-1],}, tam[:, 1:])

In [100]:

def make_dataset(pairs):
    eng_texts, tam_texts = zip(*pairs)
    eng_texts = list(eng_texts)
    tam_texts = list(tam_texts)
    dataset = tf.data.Dataset.from_tensor_slices((eng_texts, tam_texts))
    dataset = dataset.batch(batch_size)
    dataset = dataset.map(format_dataset)
    return dataset.shuffle(2048).prefetch(16).cache()

In [101]:
train_ds = make_dataset(train_pairs)
test_ds = make_dataset(test_pairs)

In [102]:
for inputs, targets in train_ds.take(1):
    print(f'inputs["encoder_inputs"].shape: {inputs["encoder_inputs"].shape}')
    print(f'inputs["decoder_inputs"].shape: {inputs["decoder_inputs"].shape}')
    print(f"targets.shape: {targets.shape}")

inputs["encoder_inputs"].shape: (64, 30)
inputs["decoder_inputs"].shape: (64, 30)
targets.shape: (64, 30)


In [103]:
class TransformerEncoder(layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.dense_dim = dense_dim
        self.num_heads = num_heads
        self.attention = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim
        )
        self.dense_proj = keras.Sequential(
            [layers.Dense(dense_dim, activation="relu"), layers.Dense(embed_dim),]
        )
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()
        self.supports_masking = True

    def call(self, inputs, mask=None):
        if mask is not None:
            padding_mask = tf.cast(mask[:, tf.newaxis, :], dtype="int32")
        attention_output = self.attention(
            query=inputs, value=inputs, key=inputs, attention_mask=padding_mask
        )
        proj_input = self.layernorm_1(inputs + attention_output)
        proj_output = self.dense_proj(proj_input)
        return self.layernorm_2(proj_input + proj_output)

In [104]:
class PositionalEmbedding(layers.Layer):
    def __init__(self, sequence_length, vocab_size, embed_dim, **kwargs):
        super().__init__(**kwargs)
        self.token_embeddings = layers.Embedding(
            input_dim=vocab_size, output_dim=embed_dim
        )
        self.position_embeddings = layers.Embedding(
            input_dim=sequence_length, output_dim=embed_dim
        )
        self.sequence_length = sequence_length
        self.vocab_size = vocab_size
        self.embed_dim = embed_dim

    def call(self, inputs):
        length = tf.shape(inputs)[-1]
        positions = tf.range(start=0, limit=length, delta=1)
        embedded_tokens = self.token_embeddings(inputs)
        embedded_positions = self.position_embeddings(positions)
        return embedded_tokens + embedded_positions

    def compute_mask(self, inputs, mask=None):
        return tf.math.not_equal(inputs, 0)

In [105]:
class TransformerDecoder(layers.Layer):
    def __init__(self, embed_dim, latent_dim, num_heads, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.latent_dim = latent_dim
        self.num_heads = num_heads
        self.attention_1 = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim
        )
        self.attention_2 = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim
        )
        self.dense_proj = keras.Sequential(
            [layers.Dense(latent_dim, activation="relu"), layers.Dense(embed_dim),]
        )
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()
        self.layernorm_3 = layers.LayerNormalization()
        self.supports_masking = True

    def call(self, inputs, encoder_outputs, mask=None):
        causal_mask = self.get_causal_attention_mask(inputs)
        if mask is not None:
            padding_mask = tf.cast(mask[:, tf.newaxis, :], dtype="int32")
            padding_mask = tf.minimum(padding_mask, causal_mask)

        attention_output_1 = self.attention_1(
            query=inputs, value=inputs, key=inputs, attention_mask=causal_mask
        )
        out_1 = self.layernorm_1(inputs + attention_output_1)

        attention_output_2 = self.attention_2(
            query=out_1,
            value=encoder_outputs,
            key=encoder_outputs,
            attention_mask=padding_mask,
        )
        out_2 = self.layernorm_2(out_1 + attention_output_2)

        proj_output = self.dense_proj(out_2)
        return self.layernorm_3(out_2 + proj_output)

    def get_causal_attention_mask(self, inputs):
        input_shape = tf.shape(inputs)
        batch_size, sequence_length = input_shape[0], input_shape[1]
        i = tf.range(sequence_length)[:, tf.newaxis]
        j = tf.range(sequence_length)
        mask = tf.cast(i >= j, dtype="int32")
        mask = tf.reshape(mask, (1, input_shape[1], input_shape[1]))
        mult = tf.concat(
            [tf.expand_dims(batch_size, -1), tf.constant([1, 1], dtype=tf.int32)],
            axis=0,
        )
        return tf.tile(mask, mult)

In [106]:
embed_dim = 256
latent_dim = 1024
num_heads = 8

encoder_inputs = keras.Input(shape=(None,), dtype="int64", name="encoder_inputs")
x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(encoder_inputs)
encoder_outputs = TransformerEncoder(embed_dim, latent_dim, num_heads)(x)
encoder = keras.Model(encoder_inputs, encoder_outputs)

In [107]:
decoder_inputs = keras.Input(shape=(None,), dtype="int64", name="decoder_inputs")
encoded_seq_inputs = keras.Input(shape=(None, embed_dim), name="decoder_state_inputs")
x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(decoder_inputs)
x = TransformerDecoder(embed_dim, latent_dim, num_heads)(x, encoded_seq_inputs)
x = layers.Dropout(0.5)(x)
decoder_outputs = layers.Dense(vocab_size, activation="softmax")(x)
decoder = keras.Model([decoder_inputs, encoded_seq_inputs], decoder_outputs)

In [108]:
decoder_outputs = decoder([decoder_inputs, encoder_outputs])
transformer = keras.Model(
    [encoder_inputs, decoder_inputs], decoder_outputs, name="transformer"
)


In [109]:
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=10,
    restore_best_weights=True
)


In [110]:
transformer.summary()

Model: "transformer"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 encoder_inputs (InputLayer  [(None, None)]               0         []                            
 )                                                                                                
                                                                                                  
 positional_embedding_10 (P  (None, None, 256)            1280768   ['encoder_inputs[0][0]']      
 ositionalEmbedding)                                      0                                       
                                                                                                  
 decoder_inputs (InputLayer  [(None, None)]               0         []                            
 )                                                                                      

In [111]:
epochs = 30
transformer.compile(
    "rmsprop", loss="sparse_categorical_crossentropy", metrics=["accuracy"]
)
transformer.fit(train_ds,
                epochs=epochs,
                validation_data=test_ds ,
                callbacks=[early_stopping] )


Epoch 1/30
625/625 [==============================] - 135s 206ms/step - loss: 7.3337 - accuracy: 0.2409 - val_loss: 6.2340 - val_accuracy: 0.3403
Epoch 2/30
625/625 [==============================] - 119s 190ms/step - loss: 6.8667 - accuracy: 0.2637 - val_loss: 5.8668 - val_accuracy: 0.3276
Epoch 3/30
625/625 [==============================] - 119s 190ms/step - loss: 6.4966 - accuracy: 0.2774 - val_loss: 5.6448 - val_accuracy: 0.3445
Epoch 4/30
625/625 [==============================] - 118s 189ms/step - loss: 6.1716 - accuracy: 0.2897 - val_loss: 5.6229 - val_accuracy: 0.3261
Epoch 5/30
625/625 [==============================] - 118s 189ms/step - loss: 5.8909 - accuracy: 0.3025 - val_loss: 5.5544 - val_accuracy: 0.3456
Epoch 6/30
625/625 [==============================] - 118s 189ms/step - loss: 5.6582 - accuracy: 0.3156 - val_loss: 5.6654 - val_accuracy: 0.3250
Epoch 7/30
625/625 [==============================] - 118s 189ms/step - loss: 5.4773 - accuracy: 0.3278 - val_loss: 5.5673 -

In [76]:
tam_vocab = tam_vectorization.get_vocabulary()
tam_index_lookup = dict(zip(range(len(tam_vocab)), tam_vocab))
max_decoded_sentence_length = 20

In [93]:
def decode_sequence(input_sentence):
    tokenized_input_sentence = eng_vectorization([input_sentence])
    decoded_sentence = "[start]"
    for i in range(max_decoded_sentence_length):
        tokenized_target_sentence = tam_vectorization([decoded_sentence])[:, :-1]
        predictions = transformer([tokenized_input_sentence, tokenized_target_sentence])

        sampled_token_index = np.argmax(predictions[0, i, :])
        sampled_token = tam_index_lookup[sampled_token_index]
        decoded_sentence += " " + sampled_token

        if sampled_token == "[end]":
            break
    return decoded_sentence

In [115]:
test_eng_texts = ""
input_sentence = ""
translated = decode_sequence(input_sentence)
print('English -' , input_sentence )
print('Tamil -' , translated)

English - you
Tamil - [start] [UNK] [end]


In [122]:

test_eng_texts = [pair[0] for pair in train_pairs]
input_sentence = random.choice(test_eng_texts)
translated = decode_sequence(input_sentence)

In [123]:
print('English -' , input_sentence )
print('Tamil -' , translated)

English - 50 lakh has been spent in
Tamil - [start] 50 லட்சம் வரை [UNK] [end]


In [129]:
!mkdir /content/drive/MyDrive/English-Tamil-Translation-Models

In [130]:
transformer.save('/content/drive/MyDrive/English-Tamil-Translation-Models/v0-50k-45M.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
